# Model using gameplay features

In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics.pairwise as sklpw


In [7]:
bggdata_df = pd.read_pickle('datasources/BGG_FINAL.pkl')

In [8]:
bgg_gameplay_df = pd.read_pickle('datasources/bgg_gameplayfeatures.pkl')

In [18]:
bgg_gameplay_df.shape

(18898, 272)

In [9]:
bgg_gameplay_df.columns

Index(['game_rank', 'gameweight', 'Abstract', 'Children's', 'Customizable',
       'Family', 'Party', 'Strategy', 'Thematic', 'Uncategorized',
       ...
       'Turn Order: Stat-Based', 'Variable Phase Order',
       'Variable Player Powers', 'Variable Setup',
       'Victory Points as a Resource', 'Voting', 'Worker Placement',
       'Worker Placement with Dice Workers',
       'Worker Placement, Different Worker Types', 'Zone of Control'],
      dtype='object', length=272)

In [122]:
bgg_gameplay_df.dropna(inplace=True)
bgg_gameplay_df.reset_index(drop=True,inplace=True)
bgg_gameplay_df.shape

(18500, 272)

### Create matrix for further use

In [49]:
allgamePLAYdocvects = np.array(bgg_gameplay_df.iloc[:,1:])

In [50]:
allgamePLAYdocvects.shape

(18500, 271)

# Using gamename

In [123]:
gamename = 'Tenzi'

In [124]:
# Get game rank from game name, and matrix index from rank
gamerank = list(bggdata_df.loc[bggdata_df['game_name']==gamename,'game_rank'])[0]
gamerank = int(gamerank)
gamerank_idx = list(bgg_gameplay_df.index[bgg_gameplay_df['game_rank']==gamerank])[0]

In [125]:
mygamePLAYvector = allgamePLAYdocvects[gamerank_idx,:] #bgg_docmatrix[gamerank_idx,:]
mygamePLAYvector = mygamePLAYvector.reshape(-1,1)

In [126]:
mysimilarities = []
for t in range(0,allgamePLAYdocvects.shape[0]):
    currgamevect = allgamePLAYdocvects[t,:]
    currgamevect = currgamevect.reshape(-1,1)
    dum = sklpw.cosine_similarity(currgamevect.T,mygamePLAYvector.T)
    mysimilarities.append(dum[0][0])
mycompletesimlist_df = pd.concat((bgg_gameplay_df['game_rank'],pd.DataFrame({'Similarity':mysimilarities})),axis=1)
mycompletesimlist_df.sort_values(by='Similarity',ascending=False,inplace=True)
mytop10simlist_df = mycompletesimlist_df[1:11]
# Create output list
urllist=[]
gamenamelist=[] # Recover from game rank
for gamerank in mytop10simlist_df['game_rank']:
    urllist.append(list(bggdata_df.loc[bggdata_df['game_rank']==str(int(gamerank)),'bgg_url'])[0])
    gamenamelist.append(list(bggdata_df.loc[bggdata_df['game_rank']==str(int(gamerank)),'game_name'])[0])
mytop10simlist_df = pd.DataFrame({'Game':gamenamelist,'Similarity':mytop10simlist_df['Similarity'],'url':urllist})
mytop10simlist_df.reset_index(drop=True,inplace=True)
mytop10simlist_df.index = mytop10simlist_df.index+1
#return mytop10simlist_df

In [127]:
mytop10simlist_df

,Game,Similarity,url
1,Mars Attacks: The Dice Game,0.838838,https://boardgamegeek.com//boardgame/155119/ma...
2,Ligretto Dice,0.836655,https://boardgamegeek.com//boardgame/55835/lig...
3,Würfel Bingo,0.832761,https://boardgamegeek.com//boardgame/29355/wur...
4,SUTAKKU,0.802895,https://boardgamegeek.com//boardgame/101796/su...
5,Bears!,0.788951,https://boardgamegeek.com//boardgame/99081/bears
6,Bohemian Villages,0.782698,https://boardgamegeek.com//boardgame/193651/bo...
7,Strike,0.778193,https://boardgamegeek.com//boardgame/123570/st...
8,Cosmic Wimpout,0.765305,https://boardgamegeek.com//boardgame/1302/cosm...
9,"Ciao, Ciao...",0.764902,https://boardgamegeek.com//boardgame/1163/ciao...
10,Untold: Adventures Await,0.764052,https://boardgamegeek.com//boardgame/226588/un...


# Make function for app:

In [ ]:
gamename='Dixit'


def getcompute_similar_by_gameplay(gamename,allgamedata_df, bgg_gameplay_df, allgamePLAYdocvects):
    # Get game rank from game name, and matrix index from rank
    gamerank = list(allgamedata_df.loc[allgamedata_df['game_name']==gamename,'game_rank'])[0]
    gamerank = int(gamerank)
    gamerank_idx = list(bgg_gameplay_df.index[bgg_gameplay_df['game_rank']==gamerank])[0]

    mygamePLAYvector = allgamePLAYdocvects[gamerank_idx,:] 
    mygamePLAYvector = mygamePLAYvector.reshape(-1,1)
    
    mysimilarities_gp = []
    for t in range(0,allgamePLAYdocvects.shape[0]):
        currgamevect = allgamePLAYdocvects[t,:]
        currgamevect = currgamevect.reshape(-1,1)
        dum = sklpw.cosine_similarity(currgamevect.T,mygamePLAYvector.T)
        mysimilarities_gp.append(dum[0][0])
    mycompletesimlist_df = pd.concat((bgg_gameplay_df['game_rank'],pd.DataFrame({'GameplaySimilarity':mysimilarities})),axis=1)
    return mycompletesimlist_df